### Proposed `summaries` Table Schema

This table will store the AI-generated summaries for each patient visit.

-   **summary_id**: A unique identifier for each summary (Primary Key).
-   **patient_id**: Links the summary to a patient.
    -   *Justification:* Your `Synthea_MVP_Cleaned_Merged.csv` file has a `PATIENT_ID` column. This confirms that summaries must be linked to a specific patient.
-   **visit_date**: The date of the doctor's visit.
    -   *Justification:* While your sample data doesn't have a clear "visit date," a real-world application absolutely requires it. This is a logical addition.
-   **source_transcription_id**: A reference to the original transcription that was summarized.
    -   *Justification:* In your `Transcriptions_MVP_Processed.csv` file, each `summary` corresponds to a `cleaned_transcription`. This field ensures we never lose that link.
-   **summary_text**: The actual summary generated by the LLM.
    -   *Justification:* This directly corresponds to the `summary` column in your `Transcriptions_MVP_Processed.csv` file. The data from that column is exactly what will be stored here.
-   **created_at**: A timestamp of when the summary was created.
    -   *Justification:* This is a standard best practice for database records.

```sql
-- SQL for the 'summaries' table
CREATE TABLE summaries (
    summary_id SERIAL PRIMARY KEY,
    patient_id VARCHAR(255) NOT NULL,
    visit_date DATE,
    source_transcription_id INT, -- Assuming transcriptions will also have their own table later
    summary_text TEXT NOT NULL,
    created_at TIMESTAMP WITH TIME ZONE DEFAULT CURRENT_TIMESTAMP
);
```

### Proposed `reminders` Table Schema

This table will track actionable reminders for medication, follow-ups, or other tasks derived from the visit summary.

-   **reminder_id**: A unique identifier for each reminder (Primary Key).
-   **patient_id**: Links the reminder to a patient.
    -   *Justification:* Same as above, all data is patient-centric.
-   **summary_id**: Links the reminder back to the summary it came from.
    -   *Justification:* The reminders are generated *from* the summary. This creates a clear relationship between a summary and its associated tasks.
-   **reminder_type**: The category of the reminder (e.g., 'Medication', 'Appointment').
    -   *Justification:* Looking at `Synthea_MVP_Cleaned_Merged.csv`, we can see data like `MEDICATION_DESC` and `CONDITION_DESC`. This implies we'll have different types of reminders.
-   **reminder_details**: The specific instruction.
    -   *Justification:* This is the most important field. The data in the `MEDICATION_DESC` column of your `Synthea_MVP_Cleaned_Merged.csv` (e.g., "Amoxicillin 500 MG Oral Tablet") is a perfect example of what will be stored in this column.
-   **due_date**: When the task or reminder is due.
    -   *Justification:* A reminder is not useful without a date or time. This is a logical requirement.
-   **status**: The current state (e.g., 'Pending', 'Completed').
    -   *Justification:* The application will need to track whether a patient has completed a reminder.

```sql
-- SQL for the 'reminders' table
CREATE TABLE reminders (
    reminder_id SERIAL PRIMARY KEY,
    patient_id VARCHAR(255) NOT NULL,
    summary_id INT REFERENCES summaries(summary_id), -- This creates a direct link to the summaries table
    reminder_type VARCHAR(50) NOT NULL, -- e.g., 'Medication', 'Appointment'
    reminder_details TEXT NOT NULL,
    due_date TIMESTAMP WITH TIME ZONE,
    status VARCHAR(50) DEFAULT 'Pending' -- Default new reminders to 'Pending'
);
```